In [7]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd


df = pd.read_csv("Car_sales_CleanData.csv")
df.rename(columns={'Company': 'Brand', 'Price ($)': 'Price'}, inplace=True)

# Aggregate sales count by Brand
df['Sales'] = df.groupby('Brand')['Brand'].transform('count')

app = dash.Dash(__name__)

In [8]:
df.head()

,Date,Customer Name,Gender,Annual Income,Dealer_Name,Brand,Model,Engine,Transmission,Color,...,Dealer_No,Body Style,Phone,Dealer_Region,Year,Month,DayOfWeek,Season,Outlier,Sales
0,2022-01-02,Geraldine,Male,13500,Buddy Storbeck's Diesel Service Inc,Ford,Expedition,DoubleÂ Overhead Camshaft,Auto,Black,...,06457-3834,SUV,8264678,Middletown,2022,1,6,Winter,False,1561
1,2022-01-02,Gia,Male,1480000,C & M Motors Inc,Dodge,Durango,DoubleÂ Overhead Camshaft,Auto,Black,...,60504-7114,SUV,6848189,Aurora,2022,1,6,Winter,False,1614
2,2022-01-02,Gianna,Male,1035000,Capitol KIA,Cadillac,Eldorado,Overhead Camshaft,Manual,Red,...,38701-8047,Passenger,7298798,Greenville,2022,1,6,Winter,False,621
3,2022-01-02,Giselle,Male,13500,Chrysler of Tri-Cities,Toyota,Celica,Overhead Camshaft,Manual,Pale White,...,99301-3882,SUV,6257557,Pasco,2022,1,6,Winter,False,1056
4,2022-01-02,Grace,Male,1465000,Chrysler Plymouth,Acura,TL,DoubleÂ Overhead Camshaft,Auto,Red,...,53546-9427,Hatchback,7081483,Janesville,2022,1,6,Winter,False,667


In [9]:
dropdown_options = [{'label': brand, 'value': brand} for brand in df['Brand'].unique()]

app.layout = html.Div([
    html.H1("Car Sales Dashboard", style={'textAlign': 'center'}),
    
    dcc.Dropdown(
        id='brand-filter',
        options=dropdown_options,
        placeholder="Select a Brand",
        multi=True
    ),
    
    dcc.Graph(id='bar-chart'),
    
    dcc.Graph(id='scatter-plot'),
    
    dcc.Graph(id='pie-chart')
])

In [10]:
@app.callback(
    Output('bar-chart', 'figure'),
    Input('brand-filter', 'value')
)
def update_bar_chart(selected_brands):
    filtered_df = df if not selected_brands else df[df['Brand'].isin(selected_brands)]
    fig = px.bar(filtered_df, x='Brand', y='Sales', title="Car Sales by Brand", color='Brand')
    return fig

@app.callback(
    Output('scatter-plot', 'figure'),
    Input('brand-filter', 'value')
)
def update_scatter_plot(selected_brands):
    filtered_df = df if not selected_brands else df[df['Brand'].isin(selected_brands)]
    fig = px.scatter(filtered_df, x='Price', y='Annual Income', color='Brand',
                     title="Price vs. Annual Income", hover_data=['Model'])
    return fig

@app.callback(
    Output('pie-chart', 'figure'),
    Input('brand-filter', 'value')
)
def update_pie_chart(selected_brands):
    filtered_df = df if not selected_brands else df[df['Brand'].isin(selected_brands)]
    fig = px.pie(filtered_df, names='Body Style', title="Car Body Style Distribution")
    return fig

# Run app
if __name__ == '__main__':
    app.run_server(debug=True)

